<a href="https://colab.research.google.com/github/HamzaWajid1/Zabaan_v1/blob/main/ZABAN_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install pytube
%pip install ibm_watson
%pip install timestamps
%pip install requests uuid
#@title Automatic Setup
%pwd
!git clone https://github.com/jaywalnut310/vits.git
!python --version
%cd vits/

!pip install Cython==0.29.21
!pip install librosa==0.8.0
!pip install phonemizer==2.2.1
!pip install scipy
!pip install numpy
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install Unidecode==1.1.1

%cd monotonic_align/
%mkdir monotonic_align
!python3 setup.py build_ext --inplace
%cd ../
%pwd
%pip install fire
%pip install commons
%pip install pydub
%pip install moviepy pydub imageio[ffmpeg]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.3/389.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 8.5 MB/s eta 0:00:00
  Created wheel for ibm_watson: filename=ibm_watson-7.0.1-py3-none-any.whl size=389784 sha256=12be7b9bef6042d16341f1a147e83f9837ba03da8d5379ddf57190e683b67132
  Stored in directory: /root/.cache/pip/wheels/34/df/f4/f8edc5ba0637dd4bfb2029741ae20402976a49d1b6bc113553
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-3.19.1-py3-none-any.whl size=95828 sha256=f95adececec8cf12c99df249b71a95

Cloning into 'vits'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 81 (delta 21), reused 21 (delta 21), pack-reused 26
Receiving objects: 100% (81/81), 3.33 MiB | 6.94 MiB/s, done.
Resolving deltas: 100% (22/22), done.
Python 3.10.12
/content/vits
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.2/974.2 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: Cython
    Found existing installation: Cython 3.0.8
    Uninstalling Cython-3.0.8:
      Successfully uninstalled Cython-3.0.8
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.9 MB/s eta 0:00:00
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201368 sha256=b3e419ac29fd657c7316431682fe0ab783e1007f40d499dc8c415aac72ca15bb
  Stored in directory: /root/.cache/pip/wheels/bf/b7/85/2f

In [2]:
import subprocess
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_videoclips
import os
import torch
torch.cuda.empty_cache()
import soundfile as sf
import shutil
import locale
locale.getpreferredencoding = lambda: "UTF-8"
from IPython.display import Audio
import re
import glob
import json
import tempfile
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import commons
import utils
import argparse
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from scipy.io.wavfile import write
import requests, uuid, json
from pytube import YouTube
import pandas as pd
from pydub import AudioSegment
import numpy as np
from moviepy.editor import VideoFileClip, AudioFileClip
import io
import soundfile as sf









In [8]:
#Meta code

#Downloading language model
def download(lang, tgt_dir="./"):
  lang_fn, lang_dir = os.path.join(tgt_dir, lang+'.tar.gz'), os.path.join(tgt_dir, lang)
  cmd = ";".join([
        f"wget https://dl.fbaipublicfiles.com/mms/tts/{lang}.tar.gz -O {lang_fn}",
        f"tar zxvf {lang_fn}"
  ])
  print(f"Download model for language: {lang}")
  subprocess.check_output(cmd, shell=True)
  print(f"Model checkpoints in {lang_dir}: {os.listdir(lang_dir)}")
  return lang_dir

#deals with special characters in specific language
def preprocess_char(text, lang=None):
    """
    Special treatement of characters in certain languages
    """
    print(lang)
    if lang == 'ron':
        text = text.replace("ț", "ţ")
    return text

#Class for reading text to generate voice
class TextMapper(object):
    def __init__(self, vocab_file):
        self.symbols = [x.replace("\n", "") for x in open(vocab_file, encoding="utf-8").readlines()]
        self.SPACE_ID = self.symbols.index(" ")
        self._symbol_to_id = {s: i for i, s in enumerate(self.symbols)}
        self._id_to_symbol = {i: s for i, s in enumerate(self.symbols)}

    def text_to_sequence(self, text, cleaner_names):
        '''Converts a string of text to a sequence of IDs corresponding to the symbols in the text.
        Args:
        text: string to convert to a sequence
        cleaner_names: names of the cleaner functions to run the text through
        Returns:
        List of integers corresponding to the symbols in the text
        '''
        sequence = []
        clean_text = text.strip()
        for symbol in clean_text:
            symbol_id = self._symbol_to_id[symbol]
            sequence += [symbol_id]
        return sequence
    #some words in punjabi cannot be translated in english , those words are replaced with roman english
    def uromanize(self, text, uroman_pl):
        iso = "xxx"
        with tempfile.NamedTemporaryFile() as tf, \
             tempfile.NamedTemporaryFile() as tf2:
            with open(tf.name, "w") as f:
                f.write("\n".join([text]))
            cmd = f"perl " + uroman_pl
            cmd += f" -l {iso} "
            cmd +=  f" < {tf.name} > {tf2.name}"
            os.system(cmd)
            outtexts = []
            with open(tf2.name) as f:
                for line in f:
                    line =  re.sub(r"\s+", " ", line).strip()
                    outtexts.append(line)
            outtext = outtexts[0]
        return outtext
    #reading the punjabi text and also normalizing it.
    def get_text(self, text, hps):
        text_norm = self.text_to_sequence(text, hps.data.text_cleaners)
        if hps.data.add_blank:
            text_norm = commons.intersperse(text_norm, 0)
        text_norm = torch.LongTensor(text_norm)
        return text_norm

    def filter_oov(self, text):
        val_chars = self._symbol_to_id
        txt_filt = "".join(list(filter(lambda x: x in val_chars, text)))
        return txt_filt

#It is basically categorizing text into roman or punjabi and also lowering the cases
def preprocess_text(txt, text_mapper, hps, uroman_dir=None, lang=None):
    txt = preprocess_char(txt, lang=lang)
    is_uroman = hps.data.training_files.split('.')[-1] == 'uroman'
    if is_uroman:
        with tempfile.TemporaryDirectory() as tmp_dir:
            if uroman_dir is None:
                cmd = f"git clone git@github.com:isi-nlp/uroman.git {tmp_dir}"
                print(cmd)
                subprocess.check_output(cmd, shell=True)
                uroman_dir = tmp_dir
            uroman_pl = os.path.join(uroman_dir, "bin", "uroman.pl")
            print(f"uromanize")
            txt = text_mapper.uromanize(txt, uroman_pl)
            print(f"uroman text: {txt}")
    txt = txt.lower()
    txt = text_mapper.filter_oov(txt)
    return txt





#IBM api key for speech to text in english.It is free
apikey = 'h3bMgb4Tldj2cqh9PDA2KieD1kprny4CpQBXp_TttbpB'
url = 'https://api.au-syd.speech-to-text.watson.cloud.ibm.com/instances/43b3559e-bba6-427b-b4c2-3a5f6a601855'






#Azure Api for translation.It is paid
key = "ded01807d1b340f9ac302e611498c392"
endpoint = "https://api.cognitive.microsofttranslator.com"

# location, also known as region.
# required if you're using a multi-service or regional (not global) resource. It can be found in the Azure portal on the Keys and Endpoint page.
location = "southeastasia"

path = '/translate'
constructed_url = endpoint + path

params = {
    'api-version': '3.0',
    'from': 'en',
    'to': ['pa']
}

headers = {
    'Ocp-Apim-Subscription-Key': key,
     # location required if you're using a multi-service or regional (not global) resource.
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}



#Acual list is storing the actual text, start list will store start time of text while end_list will store end time of text
actual_list = []
start_list=[]
end_list=[]



#Storing of punjabi text
Punjabi_list = []





#Storing generated chunks of audios as AudioSegment
generated_audios = []





# List to store the stretched audio segments
stretched_segments = []



# Function to speed up audio
def speed_up_audio(audio, speed_factor):
    # Speed up the audio
    sped_up_audio = audio.speedup(playback_speed=speed_factor)
    return sped_up_audio

# Function to create an empty audio segment
def create_empty_segment(duration):
    return AudioSegment.silent(duration=duration * 1000)  # Duration in milliseconds

# Function to add an audio segment to the project at a specific time
def add_audio_at_time(project, audio_segment, start_time, end_time):
    project = project[:start_time * 1000] + audio_segment + project[end_time * 1000:]
    return project

In [9]:
LANG = "pan"

ckpt_dir = download(LANG)




if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Run inference with {device}")
vocab_file = f"{ckpt_dir}/vocab.txt"
config_file = f"{ckpt_dir}/config.json"
assert os.path.isfile(config_file), f"{config_file} doesn't exist"
hps = utils.get_hparams_from_file(config_file)
text_mapper = TextMapper(vocab_file)
net_g = SynthesizerTrn(
    len(text_mapper.symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
net_g.to(device)
_ = net_g.eval()

g_pth = f"{ckpt_dir}/G_100000.pth"
print(f"load {g_pth}")

_ = utils.load_checkpoint(g_pth, net_g, None)





# link of the video to be downloaded
link="https://www.youtube.com/watch?v=A98wPKiPMJw"
yt = YouTube(link)

try:
    yt.streams.filter(progressive = True,
file_extension = "mp4").first().download(output_path = "/content/",
filename = "a.mp4")
except:
    print("Some Error!")
print('Task Completed!')




#Talha will need to change its path(mp4 converted to audio)
command = 'ffmpeg -i /content/a.mp4 -ab 160k -ar 44100 -vn /content/audio.wav'
subprocess.call(command, shell=True)



#Speech to text
authenticator = IAMAuthenticator(apikey)
stt = SpeechToTextV1(authenticator=authenticator)
stt.set_service_url(url)




#Talha will need to change path
#Extracting text from audio with time stamps using IBM service
with open('/content/audio.wav', 'rb') as f:
    res = stt.recognize(audio=f, content_type='audio/wav', model='en-AU_NarrowbandModel', timestamps=True).get_result()



for i in range(0,len(res['results'])):
  actual_list.append(res['results'][i]['alternatives'][0]['transcript'])
  start_list.append(res['results'][i]['alternatives'][0]['timestamps'][0][1])
  end_list.append(res['results'][i]['alternatives'][0]['timestamps'][-1][2])





# Create a list of lists
data = list(zip(actual_list, start_list, end_list))
# Create a DataFrame from the list of lists
df = pd.DataFrame(data, columns=['Text', 'start_time', 'end_time'])






#Azure translation service being used here to translate all the chunks of text
for i in range(len(df)):
  text=str(df['Text'][i])
  # You can pass more than one object in body.
  body = [{
      'text': text
  }]
  request = requests.post(constructed_url, params=params, headers=headers, json=body)
  response = request.json()
  #print(json.dumps(response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': ')))
  translated_text=response[0]['translations'][0]['text']
  Punjabi_list.append(translated_text)




df["translated"]=Punjabi_list
df['duration']=df['end_time']-df['start_time']
df['duration']=df['duration'].astype(float)







for i in range(len(Punjabi_list)):
    txt = str(df['translated'][i])
    txt = preprocess_text(txt, text_mapper, hps, lang=LANG)
    stn_tst = text_mapper.get_text(txt, hps)

    with torch.no_grad():
        x_tst = stn_tst.unsqueeze(0).to(device)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
        hyp = net_g.infer(
            x_tst, x_tst_lengths, noise_scale=.667,
            noise_scale_w=0.8, length_scale=1.0
        )[0][0,0].cpu().float().numpy()

    print(f"Generated audio")

    # Convert the generated audio array to 16-bit PCM format
    hyp_int16 = (hyp * 32767).astype(np.int16)

    # Create an AudioSegment from the NumPy array
    hyp1 = AudioSegment(
        hyp_int16.tobytes(),
        frame_rate=hps.data.sampling_rate,
        sample_width=hyp_int16.dtype.itemsize,
        channels=1
    )
    # Append the file path to the list
    generated_audios.append(hyp1)





#Talha will need to change path and delete original audio for optimization
original=AudioSegment.from_wav("/content/audio.wav")
os.remove("/content/audio.wav")




# Specify the duration of the empty audio (in seconds)
empty_duration = len(original) / 1000
# Create an empty audio segment
project = create_empty_segment(empty_duration)


# Assuming stretched is a list of AudioSegment objects
for i in range(len(df)):
    # Example usage: Replace this line with your code to get the stretched audio segment
    stretched_audio = generated_audios[i]

    # Speed up the audio segment based on the duration
    speed_factor = (len(stretched_audio)/1000) / df['duration'].iloc[i]
    stretched_audio = speed_up_audio(stretched_audio, speed_factor)

    # Append the stretched audio segment to the list
    stretched_segments.append(stretched_audio)

# Add each stretched audio segment to the project at the specified start and end times
for i, stretched_audio in enumerate(stretched_segments):
    project = add_audio_at_time(project, stretched_audio, df['start_time'].iloc[i], df['end_time'].iloc[i])





audio_segment = project

# Convert AudioSegment to MP3 format
mp3_data = audio_segment.export(format="mp3").read()

# Save the MP3 data to a temporary file
temp_mp3_path = "/content/temp_audio.mp3"
with open(temp_mp3_path, "wb") as f:
    f.write(mp3_data)

# Open the video and audio
video_clip = VideoFileClip("/content/a.mp4")
os.remove("/content/a.mp4")
audio_clip = AudioFileClip(temp_mp3_path)

# Set the audio of the video clip to the loaded audio
video_clip = video_clip.set_audio(audio_clip)

# Write the new video with the added audio to a file or display it
video_clip.write_videofile('/content/output_video.mp4', codec='libx264', audio_codec='aac')

# Cleanup: Delete the temporary MP3 file
os.remove(temp_mp3_path)






#Acual list is storing the actual text, start list will store start time of text while end_list will store end time of text
actual_list = []
start_list=[]
end_list=[]
#Storing of punjabi text
Punjabi_list = []
#Storing generated chunks of audios as AudioSegment
generated_audios = []
# List to store the stretched audio segments
stretched_segments = []


Download model for language: pan


[WARNING] /usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")



Model checkpoints in ./pan: ['config.json', 'vocab.txt', 'G_100000.pth']
Run inference with cuda
load ./pan/G_100000.pth
Task Completed!
pan
Generated audio
pan
Generated audio
pan
Generated audio
pan
Generated audio
pan
Generated audio
pan
Generated audio
Moviepy - Building video /content/output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/output_video.mp4



t: 100%|█████████▉| 7166/7190 [00:34<00:00, 249.19it/s, now=None][WARNING] /usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/a.mp4, 691200 bytes wanted but 0 bytes read,at frame 7189/7190, at time 239.87/239.89 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready /content/output_video.mp4
